Import des fonctions définies dans le module `lib` (contient les fonctions pour le processing de l'audio et la création de modèles)

In [8]:
from lib import audio_processing, modeling, prediction, save_model, load_model

Import de quelques modules complémentaires afin de visualiser les résultats et les modèles

In [9]:
from json import load
from os import listdir,system
from pathlib import Path

Pour effectuer des prédictions, on va avoir besoin de générer des spectrogrammes depuis les fichiers audio, car les modèles sont entraînés grâce à ce type de données. Techniquement, on fait appel à la fonction `audio_processing` pour cela. On est ici sur un petit exemple, on va donc ne pas utiliser le multiprocessing, en revanche l'appel à cette fonction par le `main` le prend en charge.

In [3]:
audio_folder:str = "toy_data"
output:str = "toy_train"

for specie in listdir(audio_folder):
    Path(f"{output}/{specie}").mkdir(parents=True, exist_ok=True)
    audio_processing(audio_folder,output,specie)

CRITICAL:root:Processing specie 'blujay'
C:\Users\Tharos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
C:\Users\Tharos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
CRITICAL:root:Processing specie 'bushti'
CRITICAL:root:Processing specie 'cacwre'
CRITICAL:root:Processing specie 'comyel'
CRITICAL:root:Processing specie 'haiwoo'


Afin de créer un modèle, on doit définir un certain nombre de paramètres, notamment d'architecture. Sont listés ici les paramètres d'un modèle présenté lors de la présentation orale, nommé 'modèle à 6 couches'.

In [3]:
HEIGHT: int = 500  # nb de pixels en hauteur
WIDTH: int = 400  # nb de pixels en largeur

MODEL_PARAMS: dict = {
    'model_name': '256_filtered_6L_30sp',
    'epochs': 100,
    'early_stopping': True,
    'batch': 256,
    'validation_split': 0.2,
    'layer_01_filter_count': 8,
    'layer_02_filter_count': 8,
    'layer_03_filter_count': 16,
    'layer_04_filter_count': 32,
    'layer_05_filter_count': 32,
    'layer_06_filter_count': 16,
    'layer_01_kernel_size': 5,
    'layer_02_kernel_size': 3,
    'layer_03_kernel_size': 3,
    'layer_04_kernel_size': 3,
    'layer_05_kernel_size': 3,
    'layer_06_kernel_size': 3,
    'layer_dense_size': 64,
    'dropout': 0.2,
    'num_layers': 6,
    'l1_regularization': 0.0,
    'l2_regularization': 0.01
}

On va maintenant pouvoir entraîner le modèle sur nos données que l'on vient de produire.

In [6]:
modeling(output,HEIGHT,WIDTH,MODEL_PARAMS,True,False)

Found 679 files belonging to 5 classes.
Using 544 files for training.
Found 679 files belonging to 5 classes.
Using 135 files for validation.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 500, 400, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 500, 400, 8)       608       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 250, 200, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 250, 200, 8)       584       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 125, 100, 8)      0         
 2D)                                          

C:\Users\Tharos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


3/3 [==============================] - 41s 11s/step - loss: 2.7299 - accuracy: 0.3566 - val_loss: 2.5037 - val_accuracy: 0.4815
Epoch 2/100
3/3 [==============================] - 40s 11s/step - loss: 2.5413 - accuracy: 0.4393 - val_loss: 2.3799 - val_accuracy: 0.4815
Epoch 3/100
3/3 [==============================] - 40s 11s/step - loss: 2.4138 - accuracy: 0.4393 - val_loss: 2.3044 - val_accuracy: 0.4815
Epoch 4/100
3/3 [==============================] - 40s 11s/step - loss: 2.3285 - accuracy: 0.4393 - val_loss: 2.1867 - val_accuracy: 0.4815
Epoch 5/100
3/3 [==============================] - 40s 11s/step - loss: 2.2124 - accuracy: 0.4393 - val_loss: 2.0678 - val_accuracy: 0.4815
Epoch 6/100
3/3 [==============================] - 40s 11s/step - loss: 2.0804 - accuracy: 0.4504 - val_loss: 1.9058 - val_accuracy: 0.6000
Epoch 7/100
3/3 [==============================] - 40s 11s/step - loss: 1.9357 - accuracy: 0.5276 - val_loss: 1.7402 - val_accuracy: 0.6444
Epoch 8/100
3/3 [===============

Le modèle est enregistré dans le dossier `models`, et on peut y retrouver les graphes présentant les résultats sur le jeu de validation, ainsi que l'évolution des métriques en fonction des epochs.

Des modèles légers d'exemple (entraînés sur 5 espèces) ont été sauvegardés dans le dossier `models_samples`. On va charger un de ces modèles.

In [4]:
model_path:str = "models_samples/model_01"

trained_model,classes = load_model(model_path)

On peut voir les informations d'entraînement du modèle ainsi que les paramètres ayant servi à l'entraîner, sauvegardées dans un fichier .json

In [5]:
load(open(f"{model_path}/params.json", "r"))

{'execution_time': 5623.843999999983,
 'cpu_time': 17019.21875,
 'true_epochs': 36,
 'accuracy_train': [0.25949999690055847,
  0.3269999921321869,
  0.4620000123977661,
  0.6520000100135803,
  0.7204999923706055,
  0.7639999985694885,
  0.8065000176429749,
  0.8379999995231628,
  0.8450000286102295,
  0.8514999747276306,
  0.8730000257492065,
  0.8784999847412109,
  0.8774999976158142,
  0.8880000114440918,
  0.9054999947547913,
  0.9014999866485596,
  0.9144999980926514,
  0.9200000166893005,
  0.9350000023841858,
  0.9434999823570251,
  0.949999988079071,
  0.953000009059906,
  0.949999988079071,
  0.9509999752044678,
  0.9459999799728394,
  0.9629999995231628,
  0.9649999737739563,
  0.9714999794960022,
  0.9729999899864197,
  0.9660000205039978,
  0.9750000238418579,
  0.9804999828338623,
  0.984499990940094,
  0.9810000061988831,
  0.9714999794960022,
  0.9785000085830688],
 'loss_train': [1.6012455224990845,
  1.5556195974349976,
  1.3614341020584106,
  0.9710457921028137,
  0.76

On peut également récupérer les informations de structure du modèle, sauvegardées dans un fichier texte

In [6]:
''.join(l for l in open(f"{model_path}/model.txt"))

'Model: "sequential_17"\n_________________________________________________________________\n Layer (type)                Output Shape              Param #   \n=================================================================\n rescaling_17 (Rescaling)    (None, 500, 400, 3)       0         \n                                                                 \n conv2d_84 (Conv2D)          (None, 500, 400, 8)       608       \n                                                                 \n max_pooling2d_84 (MaxPoolin  (None, 250, 200, 8)      0         \n g2D)                                                            \n                                                                 \n conv2d_85 (Conv2D)          (None, 250, 200, 8)       584       \n                                                                 \n max_pooling2d_85 (MaxPoolin  (None, 125, 100, 8)      0         \n g2D)                                                            \n                                     

On va créer un ensemble de spectrogrammes "inconnus" à partir de l'espèce 'bushti'.

In [10]:
audio_folder:str = "toy_unknown"
output:str = "predict"

for specie in listdir(audio_folder):
    Path(f"{output}/{specie}").mkdir(parents=True, exist_ok=True)
    audio_processing(audio_folder,output,specie)

system(f"mv {output}/{specie}/* {output}/")
system(f"rm -r {output}/{specie}/")

CRITICAL:root:Processing specie 'toy_prediction'
C:\Users\Tharos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


0

On va exécuter les prédictions sur les spectrogrammes que l'on vient de générer avec le modèle d'exemple.

In [15]:
for img in listdir(output):
    print(sorted(prediction(f"{output}/{img}",trained_model,HEIGHT,WIDTH,classes),key=lambda x:x[1])[::-1])

1/1 [==============================] - 0s 39ms/step
[('Cactus Wren', 14.884757995605469), ('Common Yellowthroat', 14.884757995605469), ('Hairy Woodpecker', 14.884757995605469), ('Blue Jay', 14.884759485721588), ('American Bushtit', 40.46096205711365)]
1/1 [==============================] - 0s 40ms/step
[('Blue Jay', 14.884759485721588), ('Cactus Wren', 14.884759485721588), ('Common Yellowthroat', 14.884759485721588), ('Hairy Woodpecker', 14.884759485721588), ('American Bushtit', 40.460970997810364)]
1/1 [==============================] - 0s 38ms/step
[('Blue Jay', 14.884759485721588), ('Cactus Wren', 14.884759485721588), ('Common Yellowthroat', 14.884759485721588), ('Hairy Woodpecker', 14.884759485721588), ('American Bushtit', 40.460970997810364)]
1/1 [==============================] - 0s 38ms/step
[('Blue Jay', 14.884759485721588), ('Cactus Wren', 14.884759485721588), ('Common Yellowthroat', 14.884759485721588), ('Hairy Woodpecker', 14.884759485721588), ('American Bushtit', 40.4609709